In [22]:
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

In [54]:
 file=pd.read_csv("Hard Red Winter Wheat Futures Historical Data.csv")
 df=pd.DataFrame(file)
 df

,Date,Price,Open,High,Low,Vol.,Change %
0,"Jan 15, 2021",643.00,637.75,660.00,635.50,24.80K,1.02%
1,"Jan 14, 2021",636.50,625.75,640.75,621.00,21.51K,1.72%
2,"Jan 13, 2021",625.75,622.00,639.50,617.75,25.85K,0.52%
3,"Jan 12, 2021",622.50,596.25,626.50,594.00,29.05K,4.80%
4,"Jan 11, 2021",594.00,594.50,604.75,588.00,19.84K,-0.13%
...,...,...,...,...,...,...,...
1509,"Jan 22, 2015",564.75,570.25,578.25,564.00,13.12K,-1.31%
1510,"Jan 21, 2015",572.25,578.00,583.25,570.75,14.04K,-0.95%
1511,"Jan 20, 2015",577.75,576.75,579.75,564.75,14.52K,0.13%
1512,"Jan 16, 2015",577.00,577.00,584.75,574.25,14.51K,0.70%


In [55]:
price=df.copy()
price=price.drop(["Open","High","Low","Vol.","Change %"],axis=1)
price=price.rename(columns={"Price":"Close"})
currentPrice=price["Close"][0]

price=price.drop([0],axis=0)
price["Log Return"]=pd.Series(dtype="float64")
i=1
for i in range(i,len(price["Close"])):
  price["Log Return"][i]=np.log(price["Close"][i]/price["Close"][i+1])
price


,Date,Close,Log Return
1,"Jan 14, 2021",636.50,0.017033
2,"Jan 13, 2021",625.75,0.005207
3,"Jan 12, 2021",622.50,0.046864
4,"Jan 11, 2021",594.00,-0.001262
5,"Jan 08, 2021",594.75,-0.006285
...,...,...,...
1509,"Jan 22, 2015",564.75,-0.013193
1510,"Jan 21, 2015",572.25,-0.009565
1511,"Jan 20, 2015",577.75,0.001299
1512,"Jan 16, 2015",577.00,0.006957


In [56]:
price["Average Return"]=pd.Series(dtype="float64")
price["Return STDEV"]=pd.Series(dtype="float64")
length=int(input("How Many Data Points Do You Want Included:"))
j=1
for j in range(j,len(price["Log Return"])):
  price["Average Return"]=price["Log Return"].rolling(length).mean()
  price["Return STDEV"]=price["Log Return"].rolling(length).std()
price
#1:23
expected_range=price[["Average Return","Return STDEV"]].copy()
m=1
for i in range(m,length-1):
  expected_range=expected_range.drop(index=i)
expected_range=expected_range.reset_index().drop(["index"],axis=1).drop([0],axis=0)
expected_range

How Many Data Points Do You Want Included:1200


,Average Return,Return STDEV
1,0.000294,0.017829
2,0.000286,0.017824
3,0.000252,0.017854
4,0.000224,0.017807
5,0.000212,0.017813
...,...,...
310,-0.000251,0.017643
311,-0.000263,0.017644
312,-0.000254,0.017642
313,-0.000232,0.017635


In [57]:
days=int(input("Project How Days From Now:"))

Project How Days From Now:25


In [58]:
sigma=float(input("What Standard Deviation Move:"))

What Standard Deviation Move:1


In [59]:
expected_range["N-Average Return"]=expected_range["Average Return"]*days
expected_range["N-Return STDEV"]=expected_range["Return STDEV"]*np.sqrt(days)
expected_range

,Average Return,Return STDEV,N-Average Return,N-Return STDEV
1,0.000294,0.017829,0.007351,0.089147
2,0.000286,0.017824,0.007160,0.089121
3,0.000252,0.017854,0.006302,0.089272
4,0.000224,0.017807,0.005588,0.089035
5,0.000212,0.017813,0.005296,0.089063
...,...,...,...,...
310,-0.000251,0.017643,-0.006284,0.088215
311,-0.000263,0.017644,-0.006582,0.088222
312,-0.000254,0.017642,-0.006345,0.088211
313,-0.000232,0.017635,-0.005800,0.088174


In [60]:
expected_range["Upper Bound"]=currentPrice*np.exp(expected_range["N-Average Return"]+(sigma*expected_range["N-Return STDEV"]))
expected_range["Lower Bound"]=currentPrice*np.exp(expected_range["N-Average Return"]-(sigma*expected_range["N-Return STDEV"]))
expected_range["Date"]=price["Date"]
expected_range

,Average Return,Return STDEV,N-Average Return,N-Return STDEV,Upper Bound,Lower Bound,Date
1,0.000294,0.017829,0.007351,0.089147,708.140912,592.498964,"Jan 14, 2021"
2,0.000286,0.017824,0.007160,0.089121,707.987062,592.401153,"Jan 13, 2021"
3,0.000252,0.017854,0.006302,0.089272,707.486584,591.804097,"Jan 12, 2021"
4,0.000224,0.017807,0.005588,0.089035,706.814298,591.521361,"Jan 11, 2021"
5,0.000212,0.017813,0.005296,0.089063,706.627442,591.331979,"Jan 08, 2021"
...,...,...,...,...,...,...,...
310,-0.000251,0.017643,-0.006284,0.088215,697.899910,585.020299,"Oct 23, 2019"
311,-0.000263,0.017644,-0.006582,0.088222,697.697086,584.841751,"Oct 22, 2019"
312,-0.000254,0.017642,-0.006345,0.088211,697.854211,584.986350,"Oct 21, 2019"
313,-0.000232,0.017635,-0.005800,0.088174,698.209250,585.326541,"Oct 18, 2019"


In [61]:
price=price.drop(["Average Return","Return STDEV"],axis=1)
price

,Date,Close,Log Return
1,"Jan 14, 2021",636.50,0.017033
2,"Jan 13, 2021",625.75,0.005207
3,"Jan 12, 2021",622.50,0.046864
4,"Jan 11, 2021",594.00,-0.001262
5,"Jan 08, 2021",594.75,-0.006285
...,...,...,...
1509,"Jan 22, 2015",564.75,-0.013193
1510,"Jan 21, 2015",572.25,-0.009565
1511,"Jan 20, 2015",577.75,0.001299
1512,"Jan 16, 2015",577.00,0.006957


In [62]:
final_data=price.join(expected_range.set_index("Date"),on="Date")
final_data

,Date,Close,Log Return,Average Return,Return STDEV,N-Average Return,N-Return STDEV,Upper Bound,Lower Bound
1,"Jan 14, 2021",636.50,0.017033,0.000294,0.017829,0.007351,0.089147,708.140912,592.498964
2,"Jan 13, 2021",625.75,0.005207,0.000286,0.017824,0.007160,0.089121,707.987062,592.401153
3,"Jan 12, 2021",622.50,0.046864,0.000252,0.017854,0.006302,0.089272,707.486584,591.804097
4,"Jan 11, 2021",594.00,-0.001262,0.000224,0.017807,0.005588,0.089035,706.814298,591.521361
5,"Jan 08, 2021",594.75,-0.006285,0.000212,0.017813,0.005296,0.089063,706.627442,591.331979
...,...,...,...,...,...,...,...,...,...
1509,"Jan 22, 2015",564.75,-0.013193,NaN,NaN,NaN,NaN,NaN,NaN
1510,"Jan 21, 2015",572.25,-0.009565,NaN,NaN,NaN,NaN,NaN,NaN
1511,"Jan 20, 2015",577.75,0.001299,NaN,NaN,NaN,NaN,NaN,NaN
1512,"Jan 16, 2015",577.00,0.006957,NaN,NaN,NaN,NaN,NaN,NaN


In [63]:
def phi(x):
  return np.exp(-x*x/2.0)/np.sqrt(np.pi*2)

In [64]:
def Phi(z):
    total = 0.0
    term = z
    i = 3
    while total != total + term:
        total += term
        term *= z * z / float(i)
        i += 2
    if z < -8.0: return 0.0
    if z >  8.0: return 1.0
    else: return 0.5 + total * phi(z)


In [65]:
def cdf(z,mu=0.0,sigma=1.0):
  return Phi((z-mu)/sigma)

In [66]:
probability=100*(1-(2*(1-cdf(sigma))))
probability=round(probability,1)
prob_str=str(probability)
upper_bound=str(round(final_data["Upper Bound"][1],2))
lower_bound=str(round(final_data["Lower Bound"][1],2))
days_str=str(days)

In [68]:
print(days_str+" days from January 14, 2021 there is a "+prob_str+"% chance of Kansas City Wheat being between "+upper_bound+" & "+lower_bound+".")

25 days from May 09, 2021 there is a 68.3% chance of Kansas City Wheat being between 708.14 & 592.5.
